In [2]:
from __future__ import print_function
import os.path, copy, numpy as np, time, sys
from numba import jit
#from sklearn.utils.linear_assignment_ import linear_assignment
from scipy.optimize import linear_sum_assignment as linear_assignment
from filterpy.kalman import KalmanFilter
from scipy.spatial import ConvexHull
import json
# load the nuscense library
from nuscenes import NuScenes
from nuscenes.eval.common.data_classes import EvalBoxes
from nuscenes.eval.tracking.data_classes import TrackingBox 
from nuscenes.eval.detection.data_classes import DetectionBox
from pyquaternion import Quaternion
from tqdm import tqdm

# load the self libaray 
from utils.dict import *
from utils.utils import mkdir_if_missing
from utils.utils import format_sample_result
from utils.association import associate_detections_to_trackers
from AB3DMOT import AB3DMOT
%load_ext autoreload
%autoreload 2

In [22]:
# python main.py test 0 iou 0.1 h false nuscenes results/000001; 
data_split = 'test'
covariance_id = 0
match_distance = 'iou'
match_threshold = 0.1
match_algorithm = 'h'
use_angular_velocity = False
dataset = 'nuscenes'
save_root = os.path.join('' + 'results/000001')

In [23]:
save_dir = os.path.join(save_root, data_split)
print(save_dir)
mkdir_if_missing(save_dir)
detection_file = os.path.join(detection_path , 'megvii_test.json')
data_root      = os.path.join(nuscense_path , 'test')
version='v1.0-test'
output_path = os.path.join(save_dir, 'results_test_probabilistic_tracking.json')

nusc = NuScenes(version=version, dataroot=data_root, verbose=True)
result = {}
total_time = 0.0
total_frames = 0

results/000001/test
Loading NuScenes tables for version v1.0-test...
23 category,
8 attribute,
4 visibility,
0 instance,
12 sensor,
1800 calibrated_sensor,
462901 ego_pose,
15 log,
150 scene,
6008 sample,
462901 sample_data,
0 sample_annotation,
4 map,
Done loading in 8.050 seconds.
Reverse indexing ...
Done reverse indexing in 1.9 seconds.


In [5]:
with open(detection_file) as f: 
        data = json.load(f)

In [6]:
assert 'results' in data, 'Error: No field `results` in result file. Please note that the result format changed.' \
    'See https://www.nuscenes.org/object-detection for more information.'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
all_results = EvalBoxes.deserialize(data['results'], DetectionBox)

In [8]:
print("Length of the result, ", len(all_results)) 
meta = data['meta']
print('meta: ', meta)
print("Loaded results from {}. Found detections for {} samples.".format(detection_file, len(all_results.sample_tokens)))

Length of the result,  6008
meta:  {'use_camera': False, 'use_lidar': True, 'use_radar': False, 'use_map': False, 'use_external': False}
Loaded results from /media/liangxu/ArmyData/nuscenes/Tracking_result/megvii_test.json. Found detections for 6008 samples.


In [9]:
all_results

EvalBoxes with 491560 boxes across 6008 samples

In [10]:


results = {}
processed_scene_tokens = set()


In [27]:
for sample_token_idx in tqdm(range(len(all_results.sample_tokens)-1)):
    sample_token = all_results.sample_tokens[sample_token_idx]
    scene_token = nusc.get('sample', sample_token)['scene_token']
    if scene_token in processed_scene_tokens:
        #print("This should not happed, the same token name in the detection")
        continue
    first_sample_token = nusc.get('scene', scene_token)['first_sample_token']
    current_sample_token = first_sample_token
    mot_trackers = {tracking_name: AB3DMOT(covariance_id, 
                                           tracking_name=tracking_name, 
                                           use_angular_velocity=use_angular_velocity, 
                                           tracking_nuscenes=True) for tracking_name in NUSCENES_TRACKING_NAMES}
    #print("Number of AB3DMOT Tracking", len(mot_trackers))
    while current_sample_token != '':
        results[current_sample_token] = []
        dets = {tracking_name: [] for tracking_name in NUSCENES_TRACKING_NAMES}
        info = {tracking_name: [] for tracking_name in NUSCENES_TRACKING_NAMES}
        for box in all_results.boxes[current_sample_token]:
            if box.detection_name not in NUSCENES_TRACKING_NAMES:
                continue
            q = Quaternion(box.rotation)
            #print(q.angle)
            angle = q.angle if q.axis[2] > 0 else -q.angle
            #print('box.rotation,  angle, axis: ', box.rotation, q.angle, q.axis)
            #print('box.rotation,  angle, axis: ', q.angle, q.axis)
            #[h, w, l, x, y, z, rot_y]
            detection = np.array([
              box.size[2], box.size[0], box.size[1], 
              box.translation[0],  box.translation[1], box.translation[2],
              angle])
            #print('detection: ', detection)
            information = np.array([box.detection_score])
            dets[box.detection_name].append(detection)
            info[box.detection_name].append(information)
            
        dets_all = {tracking_name: {'dets': np.array(dets[tracking_name]), 'info': np.array(info[tracking_name])}
            for tracking_name in NUSCENES_TRACKING_NAMES}
        total_frames += 1
        start_time = time.time()
        # starting the tracking
        for tracking_name in NUSCENES_TRACKING_NAMES:
            if dets_all[tracking_name]['dets'].shape[0] > 0: 
                trackers = mot_trackers[tracking_name].update(dets_all[tracking_name],
                                                              match_distance,
                                                              match_threshold,
                                                              match_algorithm, 
                                                              scene_token)
                # TODO: Need to finish the tracking update algorithm
                for i in range(trackers.shape[0]):
                    sample_result = format_sample_result(current_sample_token, 
                                                         tracking_name,
                                                         trackers[i])
                    results[current_sample_token].append(sample_result)
        cycle_time = time.time() - start_time
        total_time += cycle_time
            
        # get next frame and continue the while loop
        current_sample_token = nusc.get('sample', current_sample_token)['next']
        
    # left while loop and mark this scene as processed
    processed_scene_tokens.add(scene_token)
# finished tracking all scenes, write output data
output_data = {'meta': meta, 'results': results}
with open(output_path, 'w') as outfile:
    json.dump(output_data, outfile)
            
        
        
        

100%|██████████| 6007/6007 [01:52<00:00, 53.58it/s]  


In [ ]:
trks = np.zeros((10,7))